# Monte Carlo Control from scratch in Python and solving Frozen Lake problem.

In this notebook you will:  

1. Implement on-policy first-visit Monte Carlo Control with $\epsilon$-greedy action selection.  
2. Run MC Control to solve [Frozen Lake problem](https://gym.openai.com/envs/FrozenLake-v0/).
3. Evaluate MC Control on a Frozen Lake problem.

# Packages

For this tutorial we will use:
1. **gym**: this library is a collection of test problems — environments — that you can use to work out your reinforcement learning algorithms, we are using Frozen Lake environment for this tutorial.
2. **numpy**: the fundamental package for scientific computing with Python;
4. **bokeh**: library for creating interactive visualizations;
5. **time**: module that provides various time-related functions;

*To run this notebook, make sure you have installed these packages with versions listed in dependencies.txt file included in this repository.*

In [1]:
import gym
import numpy as np
import time

from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

env_name = 'FrozenLake-v1'

Loading BokehJS ...

Below we provide you with helper function `render_single` in order to run an instance of the the *FrozenLake-v0* environment for maximum `max_steps` timesteps, rendering the environment at each step.  

In [2]:
def render_single(env, policy, max_steps=100):
    """
    Renders policy for an environment.

    Parameters
    ----------
    env:    gym.core.Environment, open gym environment object
    policy: np.array of shape [env.nS], the action to take at a given state
    """
    episode_reward = 0
    ob = env.reset()
    for t in range(max_steps):
        env.render()
        time.sleep(0.25)
        a = policy[ob]
        ob, reward, done, _ = env.step(a)
        episode_reward += reward
        if done:
            break
    env.render();
    if not done:
        print("The agent didn't reach a terminal state in {} steps.".format(max_steps))
    else:
        print("Episode reward: %f" % episode_reward)

# MC Control Implementation

We will create class called MC control. You need to finish implementation of following methods to make it work:
1. `init_agent`
2. `evaluate_policy`
3. `argmax`
4. `get_epsilon_greedy_action`
5. `improve_policy`


Parts that you need to implement are depicted with following block :
```
# --------------------------
# <Implementation isntructions>:
# your code here (x lines)

# --------------------------
```


Below the code with `MCControl` class you will find tests, run them to make make sure that your implementation is correct.

<font color='red'>**LET'S GET STARTED!!!**</font>

**I. First let's revise the complete pseudocode for MC control:**

<font color='blue'>**Monte Carlo Control Pseudocode**:<font>
    
    
Input:  $epsilon$, $gamma$, $n\_episodes$


Initialize for all $s\in S$ and $a\in A$:    
>$Q(s, a)$ <- arbitrary  
    $\pi(s)$ <- arbitrary

Repeat for $n\_episodes$:  
>generate episode following $\epsilon$-greedy policy  
    $Q(s, a)$ <- evaluate policy using first-visit MC method   
    $\pi$ <- improve policy greedily

 
$Q^*(s, a)$ <- $Q(s, a)$  
$\pi^*$  <- $\pi$ 

**II. Here are some hints and notes on each method you need to implement:**

1. `init_agent`: 

* In this implementation we are using first time visit approach and to keep track of visits we create `visit_count` class attribute.
* You can initialise action value function arbitrariliy, but to pass test, initialise values to 0

2. `evaluate_policy`:  

For efficiency we use incremental mean approach to update action value function $Q(s, a)$ after each episode: 

$N(s, a) = N(s, a) + 1$  
$Q(s, a) = Q(s, a) + \dfrac{1}{N(s, a)}(G_t - Q(s, a)$,  
where $N(s, a)$ is visit count


3. `argmax`:  

Use greedy approach to improve policy after each episode -> choose an action with a maximum $Q$ value.


4. `get_epsilon_greedy_action`

Choose an action per state that has maximal estimated action value most of the time, but with probability epsilon ε select an action at random.


5. `improve_policy`

Improve policy using epsilon greedy approach. 

In [ ]:
class MCControl:
    '''Implements Monte Carlo Control.'''
    def __init__(self, env, num_states, num_actions, epsilon, gamma):
        '''Parameters
        ----------
        env:         gym.core.Environment, open gym environment object
        num_states:  integer, number of states in the environment
        num_actions: integer, number of possible actions
        epsilon:     float, the epsilon parameter used for exploration
        gamma:       float, discount factor
        '''
        self.env = env
        self.num_states = num_states
        self.num_actions = num_actions
        self.epsilon = epsilon
        self.gamma = gamma
        
    def run_mc_control(self, num_episodes, verbose=True):
        '''Performs Monte Carlo control task.
        
        Parameters
        ----------
        num_episodes: integer, number of episodes to run to train RL agent
        
        Returns
        ----------
        self.Q:              nested dictionary {state: {action: q value}}, final action value function
        self.policy:         list of integers of length self.num_states, final policy
        rewards_per_episode: numpy array of rewards collected at each episode
        '''
        self.init_agent()
        
        rewards_per_episode = np.array([None] * num_episodes)
        episode_len = np.array([None] * num_episodes)

        for episode in range(num_episodes):
            state_action_reward = self.generate_episode(self.policy)
            G = self.calculate_returns(state_action_reward)
            self.evaluate_policy(G)
            self.improve_policy()
            
            # Logging rewards and episode length
            total_return = 0
            for _, _, reward in state_action_reward:
                total_return += reward
            rewards_per_episode[episode] = total_return
            episode_len = len(state_action_reward)
        
        # Once training is finished, calculate final policy using argmax approach
        final_policy = self.argmax(self.Q, self.policy)
            
        if verbose:
            print (f"Finished training RL agent for {num_episodes} episodes!")
        
        
        return self.Q, final_policy, rewards_per_episode, episode_len

    def init_agent(self):
        '''Initializes RL agent components:
        self.policy:      list of integers of length self.num_states, the action to take at a given state
        self.Q:           nested dictionary {state: {action: q value}}, action value function
        self.visit_count: nested dictionary {state: {action: count}}, keeps track of how many episodes
                          state and action pair were visited for a first time in every episode
        '''
        # --------------------------
        # Randomly initialize policy, use numpy random.choice method:
        # your code here (1 line)
        self.policy = np.random.choice(num_actions, num_states)
        # --------------------------

        self.Q = {}
        self.visit_count = {}

        for state in range(self.num_states):
            self.Q[state] = {}
            self.visit_count[state] = {}
            for action in range(self.num_actions):
                # --------------------------
                # Initalize action value (self.Q) and visit count (self.visit_count) dictionaries to zero:
                # your code here (~ 2 lines)
                
                # --------------------------

    def generate_episode(self, policy):
        '''Generates episode given current policy.
        
        Parameters
        ----------
        policy: list of integers of length self.num_states, the action to take at a given state
        
        Returns
        ----------
        state_action_reward: list of tuple (state, action, reward)
        '''
        G = 0
        s = env.reset()
        a = policy[s]

        state_action_reward = [(s, a, 0)]
        while True:
            s, r, terminated, _ = env.step(a)
            if terminated:
                state_action_reward.append((s, None, r))
                break
            else:
                a = policy[s]
                state_action_reward.append((s, a, r))

        return state_action_reward
    
    def calculate_returns(self, state_action_reward):
        '''Calculates and returns total discounted reward for each pair of (s, a) appearing in the episode.
        
        Parameters
        ----------
        state_action_reward: list of tuple (state, action, reward)
        
        Returns
        ----------
        G: nested dictionary {state: {action: count}}, contains returns for every pair of (s, a) appearing in the episode
        
        '''
        G = {}
        t = 0
        for state, action, reward in state_action_reward:
            if state not in G:
                G[state] = {action: 0}
            else:
                if action not in G[state]:
                    G[state][action] = 0
            for s in G.keys():
                for a in G[s].keys():
                    G[s][a] += reward * gamma ** t
            t += 1
                
        return G
    
    def evaluate_policy(self, G):
        '''Evaluates current policy using incremental mean and updates action value function self.Q.

        Parameters
        ----------
        G: float, episode return (total discounted reward)
        state_action_reward: list of tuple (state, action, reward)
        '''
        
        for state in G.keys():
            for action in G[state].keys():
                if action:
                    # your code here (2 lines): increment self.visit_count and update self.Q for state and action pair
                    
                    # --------------------------
                
    def improve_policy(self):
        '''Improves and updates current policy self.policy using epsilon greedy approach.'''
        # Your code here (~3 lines): 
        # first use argmax method to choose actions greedily: self.policy = argmax
        # then replace greedy policy by epsilon greedy approach: self.policy[state] for every state in S = ? 
        
        # --------------------------

    def argmax(self, Q, policy):
        """
        Finds and returns greedy policy.

        Parameters
        ----------
        Q: nested dictionary {state: {action: q value}}, action value function
        policy: list of integers of length self.num_states containing last actions per state
        
        Returns
        ----------
        next_policy: list of integers of length self.num_states containing next actions with a highest value per state 

        """
        next_policy = policy
        
        for state in range(self.num_states):
            best_action = None
            best_value = float('-inf')
            # --------------------------
            # Find greedy action to take in every state and assign to policy[state]:
            # your code here (~ 5 lines)
            
            # --------------------------

        return next_policy
    
    def get_epsilon_greedy_action(self, greedy_action):
        '''Returns next action using epsilon greedy approach.
        
        Parameters
        ----------
        greedy_action: integer, greedy action (action with a maximum Q value)
        
        Returns
        ----------
        next_action: integer, either greedy or random action
        '''   
        prob = np.random.random()

        if prob < 1 - self.epsilon:
            # your code here (1 line)
            # return ?
            
            # --------------------------
        
        # your code here (1 line)
        # return ?
        
        # --------------------------

# Tests

1. Test `init_agent`

In [4]:
np.random.seed(1)

epsilon = 0.4
gamma = 0.9
n_episodes = 10000

env = None
num_states = 2
num_actions = 3

mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)

mc_model.init_agent()
assert np.all(mc_model.policy == np.array([1, 0]))
assert mc_model.Q == {0: {0: 0, 1: 0, 2: 0}, 1: {0: 0, 1: 0, 2: 0}}
assert mc_model.visit_count == {0: {0: 0, 1: 0, 2: 0}, 1: {0: 0, 1: 0, 2: 0}}

2. Test `generate_episode`

In [5]:
np.random.seed(1)

epsilon = 0.4
gamma = 0.9
n_episodes = 10000

env = gym.make(env_name)
env.seed(2)
num_states = env.observation_space.n
num_actions = env.action_space.n


mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)

policy = np.array([1, 1, 1, 1, 0, 0, 2, 2, 3, 3, 1, 1, 2, 2, 3, 3])
res = mc_model.generate_episode(policy)

assert res == [
    (0, 1, 0),
    (4, 0, 0.0),
    (4, 0, 0.0),
    (8, 3, 0.0),
    (8, 3, 0.0),
    (4, 0, 0.0),
    (0, 1, 0.0),
    (4, 0, 0.0),
    (4, 0, 0.0),
    (4, 0, 0.0),
    (8, 3, 0.0),
    (4, 0, 0.0),
    (0, 1, 0.0),
    (4, 0, 0.0),
    (8, 3, 0.0),
    (8, 3, 0.0),
    (8, 3, 0.0),
    (8, 3, 0.0),
    (4, 0, 0.0),
    (8, 3, 0.0),
    (8, 3, 0.0),
    (8, 3, 0.0),
    (9, 3, 0.0),
    (5, None, 0.0)
]

3. Test `calculate_returns`

In [6]:
np.random.seed(1)

epsilon = 0.4
gamma = 0.9
n_episodes = 10000

env = gym.make(env_name)
env.seed(2)
num_states = env.observation_space.n
num_actions = env.action_space.n


mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)

state_action_reward = [(1, 2, 0), (1, 2, 1), (2, 1, 1), (3, 2, 2)]
res = mc_model.calculate_returns(state_action_reward)

assert res == {1: {2: 3.168}, 2: {1: 2.2680000000000002}, 3: {2: 1.4580000000000002}}

3. Test `evaluate_policy`

In [7]:
np.random.seed(1)

epsilon = 0.4
gamma = 1.0
n_episodes = 10000

env = gym.make(env_name)
env.seed(2)
num_states = env.observation_space.n
num_actions = env.action_space.n

mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)
mc_model.init_agent()

G = {0: {1: 1.0}, 4: {3: 0}, 1: {3: 0}}

mc_model.evaluate_policy(G)

res_Q = {0:  {0: 0, 1: 1.0, 2: 0, 3: 0},
         1:  {0: 0, 1: 0,  2: 0,  3: 0},
         2:  {0: 0, 1: 0,  2: 0,  3: 0},
         3:  {0: 0, 1: 0,  2: 0,  3: 0},
         4:  {0: 0, 1: 0,  2: 0,  3: 0},
         5:  {0: 0, 1: 0,  2: 0,  3: 0},
         6:  {0: 0, 1: 0,  2: 0,  3: 0},
         7:  {0: 0, 1: 0,  2: 0,  3: 0},
         8:  {0: 0, 1: 0,  2: 0,  3: 0},
         9:  {0: 0, 1: 0,  2: 0,  3: 0},
         10: {0: 0, 1: 0,  2: 0,  3: 0},
         11: {0: 0, 1: 0,  2: 0,  3: 0},
         12: {0: 0, 1: 0,  2: 0,  3: 0},
         13: {0: 0, 1: 0,  2: 0,  3: 0},
         14: {0: 0, 1: 0,  2: 0,  3: 0},
         15: {0: 0, 1: 0,  2: 0,  3: 0}}

assert mc_model.Q == res_Q

res_visit_count = {0:  {0: 0, 1: 1, 2: 0, 3: 0},
                   1:  {0: 0, 1: 0, 2: 0, 3: 1},
                   2:  {0: 0, 1: 0, 2: 0, 3: 0},
                   3:  {0: 0, 1: 0, 2: 0, 3: 0},
                   4:  {0: 0, 1: 0, 2: 0, 3: 1},
                   5:  {0: 0, 1: 0, 2: 0, 3: 0},
                   6:  {0: 0, 1: 0, 2: 0, 3: 0},
                   7:  {0: 0, 1: 0, 2: 0, 3: 0},
                   8:  {0: 0, 1: 0, 2: 0, 3: 0},
                   9:  {0: 0, 1: 0, 2: 0, 3: 0},
                   10: {0: 0, 1: 0, 2: 0, 3: 0},
                   11: {0: 0, 1: 0, 2: 0, 3: 0},
                   12: {0: 0, 1: 0, 2: 0, 3: 0},
                   13: {0: 0, 1: 0, 2: 0, 3: 0},
                   14: {0: 0, 1: 0, 2: 0, 3: 0},
                   15: {0: 0, 1: 0, 2: 0, 3: 0}}
assert mc_model.visit_count == res_visit_count

4. Test `argmax`

In [8]:
np.random.seed(1)

epsilon = 0.4
gamma = 0.9
n_episodes = 10000

env = gym.make(env_name)
env.seed(2)
num_states = env.observation_space.n
num_actions = env.action_space.n

mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)

Q = {0: {0: 0, 1: 1.0, 2: 0, 3: 0},
     1: {0: 0, 1: 0, 2: 0, 3: 1.0},
     2: {0: 0, 1: 0, 2: 0, 3: 0},
     3: {0: 0, 1: 0, 2: 0, 3: 0},
     4: {0: 0, 1: 0, 2: 0, 3: 1.0},
     5: {0: 0, 1: 0, 2: 0, 3: 0},
     6: {0: 0, 1: 0, 2: 0, 3: 0},
     7: {0: 0, 1: 0, 2: 0, 3: 0},
     8: {0: 0, 1: 0, 2: 0, 3: 0},
     9: {0: 0, 1: 0, 2: 0, 3: 0},
     10: {0: 0, 1: 0, 2: 0, 3: 0},
     11: {0: 0, 1: 0, 2: 0, 3: 0},
     12: {0: 0, 1: 0, 2: 0, 3: 0},
     13: {0: 0, 1: 0, 2: 0, 3: 0},
     14: {0: 0, 1: 0, 2: 0, 3: 0},
     15: {0: 0, 1: 0, 2: 0, 3: 0}}
policy = np.array([1, 3, 0, 0, 3, 1, 3, 1, 3, 0, 0, 1, 0, 3, 1, 0])
next_policy = mc_model.argmax(Q, policy)

assert np.all(next_policy == np.array([1, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

# Take MC Control to a Frozen Lake ⛸

Now is the time to see your MC in action! Run cells below to check if it can find the way to the **Goal**.

In [9]:
np.random.seed(1)

epsilon = 0.8
gamma = 1.0
n_episodes = 1000

env = gym.make(env_name, is_slippery=False)
env.seed(0)
num_states = env.observation_space.n
num_actions = env.action_space.n

mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)

Q, policy, _, _ = mc_model.run_mc_control(n_episodes)

Finished training RL agent for 1000 episodes!


In [ ]:
render_single(env, policy, 200)

# Evaluate your MC Control on a Frozen Lake 😎

Another way to evaluate and also debug your implementation is to plot *discounted reward versus episodes* averaged over several runs, please note that we control stochasticity of each run using gym environment object's seed attribute. If this metric goes up throughout the training, it’s a good sign:

In [11]:
np.random.seed(1)
epsilon = 0.8
gamma = 1.0
n_episodes = 400

num_runs = 25

env = gym.make(env_name, is_slippery=False)
num_states = env.observation_space.n
num_actions = env.action_space.n

# every row is the record of rewards by episide per unique run, e.g. rewards_matrix[0, 0] is the rewards obtained in episode 1 of run 1
rewards_matrix = [None] * num_runs
for i in range(len(rewards_matrix)):
    rewards_matrix[i] = [None] * n_episodes

rewards_matrix = np.array(rewards_matrix)
episode_len_matrix = np.copy(rewards_matrix)
    
for run in range(num_runs):
    env.seed(run)
    np.random.seed(run)
    mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)
    Q, policy, rewards_matrix[run], episode_len_matrix[run] = mc_model.run_mc_control(n_episodes, verbose=False)

    
avg_rewards_per_episode = rewards_matrix.mean(axis=0)
cum_reward = np.cumsum(avg_rewards_per_episode)
avg_episode_len = episode_len_matrix.mean(axis=0)

In [12]:
p = figure(
    title="Sum of rewards",
    plot_width=800,
    plot_height=250,
    x_axis_label="Episodes",
    y_axis_label="Average sum of rewards during episode"
)
p.line(range(1, n_episodes + 1), avg_rewards_per_episode, line_width=2)

show(p)

Another way to check the we get an expected behaviour is to look at *cumulative discount reward* again average over several runs:

In [13]:
p = figure(
    title="Cumulative sum of rewards",
    plot_width=800,
    plot_height=250,
    x_axis_label="Episodes",
    y_axis_label="Cumulative average sum of rewards"
)
p.line(range(1, n_episodes + 1), cum_reward, line_width=2)

show(p)

# Adding Complexity 1: Stochasticity

In [ ]:
np.random.seed(1)

epsilon = 0.8
gamma = 1.0
n_episodes = 20000

env = gym.make(env_name)
env.seed(0)
num_states = env.observation_space.n
num_actions = env.action_space.n

mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)

Q, policy, _, _ = mc_model.run_mc_control(n_episodes)

In [ ]:
render_single(env, policy, 200)

### Evaluate MC

In [ ]:
# np.random.seed(1)
epsilon = 0.8
gamma = 1.0
n_episodes = 10000

num_runs = 25

env = gym.make(env_name)
num_states = env.observation_space.n
num_actions = env.action_space.n

# every row is the record of rewards by episide per unique run, e.g. rewards_matrix[0, 0] is the rewards obtained in episode 1 of run 1
rewards_matrix = [None] * num_runs
for i in range(len(rewards_matrix)):
    rewards_matrix[i] = [None] * n_episodes

rewards_matrix = np.array(rewards_matrix)
episode_len_matrix = np.copy(rewards_matrix)
    
for run in range(num_runs):
    env.seed(run)
    np.random.seed(run)
    mc_model = MCControl(env, num_states, num_actions, epsilon, gamma)
    Q, policy, rewards_matrix[run], episode_len_matrix[run] = mc_model.run_mc_control(n_episodes, verbose=False)

    
avg_rewards_per_episode = rewards_matrix.mean(axis=0)
cum_reward = np.cumsum(avg_rewards_per_episode)
avg_episode_len = episode_len_matrix.mean(axis=0)

In [17]:
p = figure(
    title="Sum of rewards",
    plot_width=800,
    plot_height=250,
    x_axis_label="Episodes",
    y_axis_label="Average sum of rewards during episode"
)
p.line(range(1, n_episodes + 1), avg_rewards_per_episode, line_width=1)

show(p)

In [19]:
p = figure(
    title="Cumulative sum of rewards",
    plot_width=800,
    plot_height=250,
    x_axis_label="Episodes",
    y_axis_label="Cumulative average sum of rewards"
)
p.line(range(1, n_episodes + 1), cum_reward, line_width=2)

show(p)

# Adding Complexity 2: 8x8 environment

Try your algorithm with a more complex [8x8 environment](https://gym.openai.com/envs/FrozenLake8x8-v0/)